In [13]:
# Data Scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_wikipedia(language_code):
    url = f"https://{language_code}.wikipedia.org/wiki/Main_Page"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        articles = []
        for heading in soup.find_all("p", class_=""):
            articles.append(heading.text.strip())

        # response = requests.get(url, headers=headers)
        # response.raise_for_status()
        # soup = BeautifulSoup(response.content, "html.parser")

        # for heading in soup.find_all("span", class_="mw-headline"):
        #     articles.append(heading.text.strip())

        # response = requests.get(url, headers=headers)
        # response.raise_for_status()
        # soup = BeautifulSoup(response.content, "html.parser")

        # for heading in soup.find_all("span", class_="mw-headline"):
        #     articles.append(heading.text.strip())


        return articles

    except requests.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.ConnectionError as errc:
        print(f"Error Connecting: {errc}")
    except requests.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.RequestException as err:
        print(f"Other Error: {err}")
    return []



languages = {
    "en": "English",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
    "it": "Italian",
    "nl": "Dutch",
    "pt": "Portuguese",
    "ru": "Russian",
    "ja": "Japanese",
    "zh": "Chinese",
    "ar": "Arabic",
    "hi": "Hindi",
    "ko": "Korean",
    "tr": "Turkish",
    "pl": "Polish",
    "sv": "Swedish",
    "uk": "Ukrainian",
    "cs": "Czech",
    "el": "Greek",
    "he": "Hebrew",
    "th": "Thai",
    "da": "Danish",
    "fi": "Finnish",
    "id": "Indonesian",
    "no": "Norwegian",
    "hu": "Hungarian",
    "ro": "Romanian",
    "sk": "Slovak",
    "vi": "Vietnamese",
    "bn": "Bengali",
    "sr": "Serbian",
    "hr": "Croatian",
    "ms": "Malay",
    "lt": "Lithuanian",
    "sl": "Slovenian",
    "et": "Estonian",
    "gl": "Galician",
    "ta": "Tamil",
    "ca": "Catalan",
    "ur": "Urdu",
    "eu": "Basque",
    "be": "Belarusian",
    "is": "Icelandic",
    "af": "Afrikaans",
    "sw": "Swahili",
    "am": "Amharic",
    "fa": "Persian",
    "ne": "Nepali",
    "so": "Somali",
    "ka": "Georgian",
    "la": "Latin",
    "km": "Khmer",
    "mk": "Macedonian",
    "mn": "Mongolian",
    "fy": "Western Frisian",
    "gu": "Gujarati",
    "pa": "Punjabi",
    "jv": "Javanese",
    "ceb": "Cebuano",
    "ha": "Hausa",
    "yo": "Yoruba",
    "su": "Sundanese",
    "si": "Sinhala",
    "sn": "Shona",
    "hmn": "Hmong",
    "uz": "Uzbek",
    "rw": "Kinyarwanda",
    "glg": "Galician",
    "kk": "Kazakh",
    "ku": "Kurdish",
    "tt": "Tatar",
    "ast": "Asturian",
    "qu": "Quechua",
    "tl": "Tagalog",
    "tg": "Tajik",
    "my": "Burmese",
    "or": "Odia",
    "ky": "Kyrgyz",
    "et": "Estonian",
    "lt": "Lithuanian",
    "lv": "Latvian",
    "fi": "Finnish",
    "hr": "Croatian",
    "sr": "Serbian",
    "sk": "Slovak",
    "sl": "Slovenian",
    "ms": "Malay",
    "th": "Thai",
    "vi": "Vietnamese",
    "bg": "Bulgarian",
    "ro": "Romanian",
    "tr": "Turkish",
    "is": "Icelandic",
    "hu": "Hungarian",
}


data = []
for lang_code, lang_name in languages.items():
    wiki_articles = scrape_wikipedia(lang_code)
    if wiki_articles:
        for article in wiki_articles:
            data.append({"Language": lang_name, "Text": article})

# for lang_code, lang_name in languages.items():
#     wiki_articles = scrape_wikipedia(lang_code)
#     if wiki_articles:
#         for article in wiki_articles:
#             data.append({"Language": lang_name, "Text": article})


df = pd.DataFrame(data)
df.to_csv("multilingual_wikipedia_dataset.csv", index=False)


Error Connecting: HTTPSConnectionPool(host='hmn.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Main_Page (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bc84cbf4d90>: Failed to resolve 'hmn.wikipedia.org' ([Errno -2] Name or service not known)"))
Error Connecting: HTTPSConnectionPool(host='glg.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Main_Page (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7bc84ca41360>: Failed to resolve 'glg.wikipedia.org' ([Errno -2] Name or service not known)"))


In [21]:
#Data Wrangling


import pandas as pd
import pandas as pd
import re
from unicodedata import normalize



# aggregated_df = df.groupby('Language')['Text'].agg(lambda x: ' '.join(x)).reset_index()
aggregated_df=df
aggregated_df.fillna(inplace=True)

aggregated_df['Text'] = aggregated_df['Text'].str.lower()


aggregated_df['Text'] = aggregated_df['Text'].apply(lambda x: re.findall(r'\b\w+\b', x))


aggregated_df['Text'] = aggregated_df['Text'].apply(lambda x: [word for word in x if word.isalnum()])


aggregated_df['Text'] = aggregated_df['Text'].apply(lambda x: [normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8') for word in x])


aggregated_df['Text'] = aggregated_df['Text'].apply(lambda x: [word for word in x if word.isalnum()])

print(aggregated_df)

ValueError: ignored

In [ ]:
# Data Embedding
!pip install gensim


In [15]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np


word2vec_model = Word2Vec(aggregated_df['Text'], vector_size=100, window=5, min_count=1, workers=4)

def average_word_embedding(sentence):
    embeddings = [word2vec_model.wv[word] for word in sentence if word in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

aggregated_df['Word_Embedding'] = aggregated_df['Text'].apply(lambda x: average_word_embedding(x))

df.to_csv("dataset.csv", index=False)


In [16]:
print(aggregated_df)

       Language                                               Text  \
0       English  [hurricane, erika, was, a, weak, hurricane, th...   
1       English  [december, 5, krampusnacht, in, parts, of, cen...   
2       English  [columbidae, is, a, bird, family, consisting, ...   
3       English            [photograph, credit, charles, j, sharp]   
4       English  [wikipedia, is, written, by, volunteer, editor...   
...         ...                                                ...   
1260    Latvian  [vikipedija, ir, bezpelnas, organizacijas, wik...   
1261  Bulgarian                                                 []   
1262  Bulgarian                                                 []   
1263  Bulgarian                                                 []   
1264  Bulgarian                                                 []   

                                         Word_Embedding  
0     [0.0005811478, 0.00051339774, 0.0009449449, 0....  
1     [0.0011380664, -0.0002042058, 0.00096

In [29]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GlobalMaxPooling1D, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming 'aggregated_df' is your dataset DataFrame
# Assuming 'text' contains the text data for each sample
# Assuming 'Language' is the target variable

# Tokenize the text and pad sequences
max_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(aggregated_df['Text'])
vocab_size = len(tokenizer.word_index) + 1
# X = pad_sequences(tokenizer.texts_to_sequences(aggregated_df['Text']), maxlen=max_length, padding='post')
X = np.vstack(aggregated_df['Word_Embedding'])  # Assuming 'Word_Embedding' contains numpy arrays
# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(aggregated_df['Language'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Define and compile the model
embedding_dim = 100
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length, trainable=True),
    Bidirectional(LSTM(128, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),

    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)




print("Test Accuracy:", test_accuracy)

Epoch 1/2
36/36 [==============================] - 20s 407ms/step - loss: 4.3419 - accuracy: 0.0220 - val_loss: 4.2717 - val_accuracy: 0.0866
Epoch 2/2
4/4 [==============================] - 0s 87ms/step - loss: 4.1892 - accuracy: 0.0866
Test Accuracy: 0.08661417663097382
